In [1]:
from typing import List, Optional

from openai import OpenAI
from dotenv import load_dotenv
import os
import glob
from docx import Document
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import json
import io
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('max_colwidth', None)

load_dotenv()

True

In [2]:
def load_doc(path: str, add_tables: bool = False):
    doc = Document(path)
    full_text = []
    for paragraph in doc.paragraphs:
        if len(paragraph.text) > 0:
            full_text.append(paragraph.text)
    
    text = "\n".join(full_text)
    if add_tables:
        tables_str = []
        for table_index, table in enumerate(doc.tables):
            table_str = f"Таблица {table_index + 1}"
            table_data = []
            for row in table.rows:
                row_data = [cell.text for cell in row.cells]
                table_data.append(row_data)
            markup_table = pd.DataFrame(np.vstack(table_data)).to_markdown(index=False)
            table_str += "\n" + markup_table
            tables_str.append(table_str)
        tables_full_str = "\n".join(tables_str)
        text = text + "\n" + tables_full_str
    return text

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_TOKEN"))

## Load data

In [4]:
df = pd.read_excel("./data/karti competencii_marketing.xlsx", skiprows = [0])

In [5]:
def filter_func(s: str):
    num_dots = sum([letter == "." for letter in s])
    if num_dots == 1:
        return True
    return False

In [6]:
i = 1
main_competencies = {}
for value in df["Трудовые функции / компетенции"].dropna().values:
    if value.startswith(f"{i}. "):
        main_competencies[value.replace(f"{i}. ", "")] = []
        key = value.replace(f"{i}. ", "")
        i += 1
        j = 1
    if value.startswith(f"{i-1}.{j}"):
        main_competencies[key].append(value.replace(f"{i-1}.{j}. ", ""))
        j += 1

In [7]:
print("\n".join(list(main_competencies.keys())))

Сбор маркетинговой информации и проведение маркетинговых исследований
Конкурентный анализ и выстраивание стратегии отличия от конкурентов 
Ассортиментный анализ и разработка товарной политики
Анализ рынка и тенденций его развития
Работа с клиентской базой
Прогнозирование спроса и продаж
Разработка конкурентных преимуществ
Разработка маркетинговой политики компании
Ценообразование и ценовая политика 
Маркетинговые и рекламные мероприятия
Разработка рекламных и информационных материалов
Генерация контента 
Вывод на рынок нового продукта
Руководство отделом/сотрудниками
Выстраивание работы с отделом продаж
Трейд-маркетинг. Разработка, реализация и совершенствование системы распределения (дистрибуции) и сбытовой политики
Управление маркетинговой деятельностью
Поисковое продвижение в интернете 
Контекстно-медийное продвижение в интернете 
Продвижение в социальных интернет-медиа (SMM)
Реализация медийной стратегии продвижения в интернете
Проведение исследований в интернете
Управление персона

In [8]:
with open("./save/competencies.json", "w") as f:
    json.dump(main_competencies, f)

In [8]:
all_competencies = []
for key in main_competencies:
    if len(main_competencies[key]) == 0:
        main_competencies[key].append(key)
    all_competencies += main_competencies[key]

## Мэтчинг компетенций

In [9]:
text_task = load_doc("./data/долж_инструкция_маркетолог.docx")

In [133]:
def find_info(tech_task: str, query: str, examples: Optional[List] = None, query_desc: Optional[str] = None):
    question = f"Какие {query} должны быть у специалиста из должностной инструкции?"
    prompt = f"Должностная инструкция:\n{tech_task}\nВопрос: {question}\nОтвет:"
    system_prompt = f"Вы эксперт в найме персонала. Необходимо извлечь необходимые {query} из должностной инструкции, "\
                    f"как отдельное поле в json с ключом '{query}'."
    if query_desc is not None:
        system_prompt += f"\n{query_desc}"
    if examples is not None:
        examples_str = '\n'.join(examples)
        system_prompt = system_prompt + f" Примеры {query}:\n{examples_str}"
        
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format={ "type": "json_object"}
    )
    return completion.choices[0].message.content

In [136]:
query_desc = "Компетенции - это трудовые функции кандидата. То, что он должен будет делать на работе, те функци, которые будет выполнять."
info_competencies = find_info(text_task, "компетенции", examples=all_competencies, query_desc=query_desc)

In [137]:
len(json.loads(info_competencies)['компетенции'])

35

In [173]:
def compare_info(query: str, list_ref: List, list_found: List):
    question = f"Сопоставь два набора {query}"
    list_ref_str = '\n'.join(list_ref)
    list_found_str = '\n'.join(list_found)
    prompt = f"Существующие {query}:\n{list_ref_str}\nНайденные в должностной инструкции {query}:\n{list_found_str}\nЗапрос: {question}\nОтвет:"
    system_prompt = f"Вы эксперт в найме персонала. Необходимо сопоставить существующие {query} и найденные "\
                    f"{query} в должностной инструкции и вывести ответ в формате одной таблицы markup. Дополнительно ничего не нужно выводить. "\
                    f"В первой колонке таблицы должны быть {query} из существующих, во второй - {query} из найденных в должностной инструкции. "\
                    f"{query} не должны дублироваться в колонках. Соответсвие должно быть один к одному. "\
                    f"Если для {query} из должностной инструкции не найдена похожая, то нужно оставить соответсвующую ячеку в колонке существующих {query} пустой. Наоборот так же. "\
                    f"В таблице должно быть столько строк, сколько {query} в должностной инструкции!"
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ],
        # response_format={ "type": "json_object"}
    )
    return completion.choices[0].message.content

In [118]:
res = compare_info(query="компетенции", list_ref=all_competencies, list_found=json.loads(info_competencies)["компетенции"])

In [119]:
print(res)

| Существующие компетенции                                                                                   | Найденные компетенции                                                                                      |
|------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------|
| Поиск, сбор и анализ маркетинговой информации                                                              | Поиск, сбор и анализ маркетинговой информации                                                              |
| Проводить маркетинговые исследования и анализировать результаты                                            | Проводить маркетинговые исследования и анализировать результаты                                            |
|                                                                                                            | Разработк

In [120]:
df = pd.read_csv(io.StringIO(res), sep="|", skiprows=[1], skipinitialspace=True)
df = df.iloc[:, 1:-1]

In [121]:
df

,Существующие компетенции,Найденные компетенции
0,"Поиск, сбор и анализ маркетинговой информации ...","Поиск, сбор и анализ маркетинговой информации ..."
1,Проводить маркетинговые исследования и анализи...,Проводить маркетинговые исследования и анализи...
2,NaN,Разработка стратегии продвижения товаров и усл...
3,10.1 Разработка стратегии и плана проведения м...,"Организация, проведение и анализ эффективности..."
4,Создание рекламных и информационных материалов...,Разработка рекламно-информационных материалов ...
5,"Организация исследований по изучению рынка, сб...",Организация и проведение внутренних и внешних ...
6,"Анализ оперативных данных об итогах продаж, об...",Анализ продаж и разработка мер по стимулирован...
7,Разработка ценовой политики компании ...,Разработка ассортиментной и ценовой политики ...
8,Определение емкости рынка и прогнозирование пр...,Вывод на рынок новых товаров и услуг ...
9,NaN,Организация участия компании в профильных выст...


In [80]:
df_ = pd.read_csv("./data/competencies_matched.csv")

In [81]:
df_["Соответствие"].value_counts(normalize = True).to_frame()

,proportion
Соответствие,
Не найдено,0.35
Хорошо,0.35
Идеально,0.25
Норм,0.05


In [82]:
df_

,Существующие компетенции,Найденные компетенции,Соответствие,Sim_score
0,"Поиск, сбор и анализ маркетинговой информации","Поиск, сбор и анализ маркетинговой информации",Идеально,1.000
1,Проводить маркетинговые исследования и анализировать результаты,Проводить маркетинговые исследования и анализировать результаты,Идеально,1.000
2,NaN,Разработка стратегии продвижения товаров и услуг,Не найдено,0.000
3,"Разработка стратегии и плана проведения маркетинговых и рекламных мероприятий, контроль реализации и оценка эффективности","Организация, проведение и анализ эффективности маркетинговых кампаний",Хорошо,0.740
4,Создание рекламных и информационных материалов,Разработка рекламно-информационных материалов,Идеально,0.854
5,"Организация исследований по изучению рынка, сбору и хранению информации",Организация и проведение внутренних и внешних маркетинговых исследований,Хорошо,0.743
6,"Анализ оперативных данных об итогах продаж, об остатках, о дефиците",Анализ продаж и разработка мер по стимулированию сбыта,Хорошо,0.606
7,Разработка ценовой политики компании,Разработка ассортиментной и ценовой политики,Идеально,0.796
8,Определение емкости рынка и прогнозирование продаж нового продукта,Вывод на рынок новых товаров и услуг,Хорошо,0.565
9,NaN,Организация участия компании в профильных выставках,Не найдено,0.000


## Поиск соответствий с помощью ембедингов

In [52]:
def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_emb_list(texts_list, model="text-embedding-3-large"):
    res = []
    for text in tqdm(texts_list):
        emb = np.array(get_embedding(text=text))
        res.append(emb)
    return np.array(res)

In [54]:
comp_query = df_["Найденные компетенции"].values
comp_query_embs = get_emb_list(comp_query)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.13it/s]


In [57]:
comp_matched = df_["Существующие компетенции"].fillna("None").values
comp_matched_embs = get_emb_list(comp_matched)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.84it/s]


In [61]:
comp_query_embs.shape

(20, 3072)

In [83]:
sims = np.diag(cosine_similarity(comp_query_embs, comp_matched_embs)).round(3)
df_["Sim_score"] = sims

In [84]:
df_.loc[df_["Существующие компетенции"].isna(), "Sim_score"] = 0.0

In [85]:
df_

,Существующие компетенции,Найденные компетенции,Соответствие,Sim_score
0,"Поиск, сбор и анализ маркетинговой информации","Поиск, сбор и анализ маркетинговой информации",Идеально,1.000
1,Проводить маркетинговые исследования и анализировать результаты,Проводить маркетинговые исследования и анализировать результаты,Идеально,1.000
2,NaN,Разработка стратегии продвижения товаров и услуг,Не найдено,0.000
3,"Разработка стратегии и плана проведения маркетинговых и рекламных мероприятий, контроль реализации и оценка эффективности","Организация, проведение и анализ эффективности маркетинговых кампаний",Хорошо,0.740
4,Создание рекламных и информационных материалов,Разработка рекламно-информационных материалов,Идеально,0.854
5,"Организация исследований по изучению рынка, сбору и хранению информации",Организация и проведение внутренних и внешних маркетинговых исследований,Хорошо,0.743
6,"Анализ оперативных данных об итогах продаж, об остатках, о дефиците",Анализ продаж и разработка мер по стимулированию сбыта,Хорошо,0.606
7,Разработка ценовой политики компании,Разработка ассортиментной и ценовой политики,Идеально,0.796
8,Определение емкости рынка и прогнозирование продаж нового продукта,Вывод на рынок новых товаров и услуг,Хорошо,0.565
9,NaN,Организация участия компании в профильных выставках,Не найдено,0.000


In [101]:
print("\n".join(df_['Sim_score'].values.astype(str)))

1.0
1.0
0.0
0.74
0.854
0.743
0.606
0.796
0.565
0.0
0.0
0.614
0.0
0.422
0.541
0.566
0.0
0.595
0.0
0.0


In [86]:
comp_not_found = df_[df_["Sim_score"] < 0.55 ]["Найденные компетенции"].values
comp_not_found_embs = get_emb_list(comp_not_found)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.90it/s]


In [58]:
all_comp_np = get_emb_list(all_competencies)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:55<00:00,  1.75it/s]


In [39]:
# np.save("./save/all_comp_embs.npy", all_comp_np)

In [203]:
all_comp_np = np.load("./save/all_comp_embs.npy")

In [145]:
def find_match_cosine(emb_ref, emb_query, all_competencies, comp_query):
    similarities = cosine_similarity(emb_query, emb_ref)
    idices_most_similar = similarities.argmax(axis = 1)
    similarities = similarities.max(axis = 1).round(3)
    most_similar = np.array(all_competencies)[idices_most_similar]
    df_sim = pd.DataFrame(
        np.vstack([comp_query, most_similar, similarities]).T,
        columns = ["Найденные компетенции", "Существующие компетенции", "cosine_sim"]
    )
    return df_sim

In [93]:
df_sim_not_found = find_match_cosine(all_comp_np, comp_not_found_embs, all_competencies, comp_not_found)

In [94]:
df_sim_not_found

,Найденные компетенции,Существующие компетенции,cosine_sim
0,Разработка стратегии продвижения товаров и услуг,Разработка маркетинговой политики компании,0.74
1,Организация участия компании в профильных выставках,Реализация маркетинговых и рекламных мероприятий,0.493
2,Организация мероприятий,Реализация маркетинговых и рекламных мероприятий,0.59
3,Создание отчетов по результатам маркетингового исследования,Проводить маркетинговые исследования и анализировать результаты,0.641
4,Определение конкурентоспособного ассортимента товаров и услуг,Ассортиментный анализ,0.579
5,Проведение тестирования инновационных товаров и услуг,"Разработка нового продукта: идея, прототип, уточнение требований, тестирование и управление качеством",0.541
6,Организация внутренних и внешних коммуникационных кампаний,Составление медиапланов и проведение медийных кампаний,0.675
7,Анализ результативности коммуникационной политики,Оценивать эффективность и составлять отчеты по реализации контекстно-медийной стратегии продвижения в интернете,0.593
8,Работа с подрядчиками,Работа со стажерами,0.53


In [102]:
df_sim_not_found.to_csv("./save/comp_second_stage_matching.csv", index = False)

In [106]:
match_status = [
    'Хорошо',
    'Норм',
    'Хорошо',
    'Идеально',
    'Идеально',
    'Хорошо',
    'Хорошо',
    'Норм',
    'Не соответствует'
]
df_sim_not_found["Соответствие"] = match_status

In [110]:
df_match_final = pd.concat([
    df_[
        ~df_["Найденные компетенции"].isin(df_sim_not_found["Найденные компетенции"].values)
    ].rename(columns = {"Sim_score": "cosine_sim"}),
    df_sim_not_found
])

In [118]:
df_match_final['Соответствие'].value_counts(normalize=True).to_frame()

,proportion
Соответствие,
Хорошо,0.50
Идеально,0.35
Норм,0.10
Не соответствует,0.05


In [103]:
df_sim_query = find_match_cosine(all_comp_np, comp_query_embs, all_competencies, comp_query)

In [119]:
df_sim_query

,Найденные компетенции,Существующие компетенции,cosine_sim
0,"Поиск, сбор и анализ маркетинговой информации","Поиск, сбор и анализ маркетинговой информации",1.0
1,Проводить маркетинговые исследования и анализировать результаты,Проводить маркетинговые исследования и анализировать результаты,1.0
2,Разработка стратегии продвижения товаров и услуг,Разработка маркетинговой политики компании,0.74
3,"Организация, проведение и анализ эффективности маркетинговых кампаний",Реализация маркетинговых и рекламных мероприятий,0.704
4,Разработка рекламно-информационных материалов,Подготовка рекламных и информационных материалов,0.866
5,Организация и проведение внутренних и внешних маркетинговых исследований,Проводить маркетинговые исследования и анализировать результаты,0.784
6,Анализ продаж и разработка мер по стимулированию сбыта,"Анализ и корректировка рекламных кампаний, идущих через отдел продаж",0.642
7,Разработка ассортиментной и ценовой политики,Разработка ценовой политики компании,0.796
8,Вывод на рынок новых товаров и услуг,Реализация маркетинговых и рекламных мероприятий по продвижению нового продукта,0.599
9,Организация участия компании в профильных выставках,Реализация маркетинговых и рекламных мероприятий,0.493


In [114]:
df_sim_query_compare = df_sim_query.merge(df_match_final, on = "Найденные компетенции", suffixes=[" cosine", " combined"])

In [116]:
df_sim_query_compare.to_csv("./save/comp_matching_compare.csv", index = False)

# Валидация

In [128]:
text_task = load_doc("./data/долж_инструкция_директор_маркетинг.docx")

In [138]:
query_desc = "Компетенции - это трудовые функции кандидата. То, что он должен будет делать на работе, "\
"те функци, которые будет выполнять."
info_competencies_director = find_info(text_task, "компетенции", examples=all_competencies, query_desc=query_desc)

In [139]:
len(json.loads(info_competencies_director)['компетенции'])

36

In [140]:
json.loads(info_competencies_director)['компетенции']

['Разработка и контроль реализации маркетинговой стратегии компании',
 'Составление, согласование и контроль исполнения маркетингового бюджета',
 'Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынка, конкурентной среды и др.)',
 'Организация мониторинга и анализа рекламной активности конкурентов',
 'Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж',
 'Организация, контроль и анализ эффективности маркетинговых кампаний',
 'Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей',
 'Контроль разработки бренда компании',
 'Контроль разработки фирменного стиля',
 'Контроль разработки полиграфической и сувенирной продукции, рекламно-информационных материалов в соответствии со стандартами фирменного стиля',
 'Оптимизация бизнес-процессов маркетинговой деятельности компании',
 'Разработка маркетинговой стратегии компании',
 'Выстраивание системы взаим

In [142]:
comp_query_director = json.loads(info_competencies_director)['компетенции']

In [143]:
comp_query_director_embs = get_emb_list(comp_query_director)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:15<00:00,  2.25it/s]


In [204]:
df_sim_query = find_match_cosine(all_comp_np, comp_query_director_embs, all_competencies, comp_query_director)

In [205]:
df_sim_query

,Найденные компетенции,Существующие компетенции,cosine_sim
0,Разработка и контроль реализации маркетинговой стратегии компании,Разработка маркетинговой политики компании,0.768
1,"Составление, согласование и контроль исполнения маркетингового бюджета",Реализация маркетинговых и рекламных мероприятий,0.629
2,"Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынка, конкурентной среды и др.)",Проводить маркетинговые исследования и анализировать результаты,0.679
3,Организация мониторинга и анализа рекламной активности конкурентов,Аналитика по информационному полю и конкурентам,0.624
4,"Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж",Разработка ценовой политики компании,0.68
5,"Организация, контроль и анализ эффективности маркетинговых кампаний",Реализация маркетинговых и рекламных мероприятий,0.676
6,"Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей",Реализация маркетинговых и рекламных мероприятий,0.646
7,Контроль разработки бренда компании,"Разработка, контроль и корректировка стратегии проведения контекстно-медийной кампании",0.598
8,Контроль разработки фирменного стиля,Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.685
9,"Контроль разработки полиграфической и сувенирной продукции, рекламно-информационных материалов в соответствии со стандартами фирменного стиля",Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.664


In [206]:
df_sim_query.shape

(36, 3)

In [174]:
res = compare_info(query="компетенции", list_ref=all_competencies, list_found=comp_query_director)

In [175]:
print(res)

| Существующие компетенции                                                | Найденные компетенции                                                                                                               |
|------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|
|                                                                        | Разработка и контроль реализации маркетинговой стратегии компании                                                                  |
|                                                                        | Составление, согласование и контроль исполнения маркетингового бюджета                                                              |
| Проводить маркетинговые исследования и анализировать результаты       | Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынк

In [176]:
df = pd.read_csv(io.StringIO(res), sep="|", skiprows=[1], skipinitialspace=True)
df = df.iloc[:, 1:-1]

In [177]:
df.columns = [col.strip() for col in df.columns]

In [178]:
df = df.dropna(subset=["Найденные компетенции"])

In [231]:
df_ = df.rename(columns={"Найденные компетенции": "Найденные компетенции"})

In [232]:
df_

,Существующие компетенции,Найденные компетенции
0,NaN,Разработка и контроль реализации маркетинговой стратегии компании
1,NaN,"Составление, согласование и контроль исполнения маркетингового бюджета"
2,Проводить маркетинговые исследования и анализировать результаты,"Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынка, конкурентной среды и др.)"
3,Анализ конкурентной среды,Организация мониторинга и анализа рекламной активности конкурентов
4,NaN,"Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж"
5,Анализ и корректировка рекламных кампаний,"Организация, контроль и анализ эффективности маркетинговых кампаний"
6,Разработка и ведение базы данных о рынке,"Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей"
7,Создание брендов и их внедрение на рынок,Контроль разработки бренда компании
8,подг. предложений по форм. фирм. стиля,Контроль разработки фирменного стиля
9,Подготовка рекламных и информационных материалов,"Контроль разработки полиграфической и сувенирной продукции, рекламно-информационных материалов в соответствии со стандартами фирменного стиля"


In [233]:
df_["Найденные компетенции"] = df_["Найденные компетенции"].apply(lambda x: x.strip())
df_["Существующие компетенции"] = df_["Существующие компетенции"].fillna("None").apply(lambda x: x.strip())

In [212]:
comp_query = df_["Найденные компетенции"].values
comp_query_embs = get_emb_list(comp_query)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:15<00:00,  2.25it/s]


In [214]:
comp_matched = df_["Существующие компетенции"].values
comp_matched_embs = get_emb_list(comp_matched)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:13<00:00,  2.69it/s]


In [215]:
comp_query_embs.shape

(35, 3072)

In [234]:
sims = np.diag(cosine_similarity(comp_query_embs, comp_matched_embs)).round(3)
df_["Sim_score"] = sims

In [235]:
df_.loc[df_["Существующие компетенции"].isna(), "Sim_score"] = 0.0

In [187]:
df_

,Существующие компетенции,Найденные компетенции,Sim_score
0,NaN,Разработка и контроль реализации маркетинговой стратегии компании,0.000
1,NaN,"Составление, согласование и контроль исполнения маркетингового бюджета",0.000
2,Проводить маркетинговые исследования и анализировать результаты,"Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынка, конкурентной среды и др.)",0.710
3,Анализ конкурентной среды,Организация мониторинга и анализа рекламной активности конкурентов,0.649
4,NaN,"Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж",0.000
5,Анализ и корректировка рекламных кампаний,"Организация, контроль и анализ эффективности маркетинговых кампаний",0.741
6,Разработка и ведение базы данных о рынке,"Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей",0.453
7,Создание брендов и их внедрение на рынок,Контроль разработки бренда компании,0.617
8,подг. предложений по форм. фирм. стиля,Контроль разработки фирменного стиля,0.550
9,Подготовка рекламных и информационных материалов,"Контроль разработки полиграфической и сувенирной продукции, рекламно-информационных материалов в соответствии со стандартами фирменного стиля",0.607


In [218]:
comp_not_found = df_[df_["Sim_score"] < 0.55]["Найденные компетенции"].values
comp_not_found_embs = get_emb_list(comp_not_found)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.53it/s]


In [219]:
df_sim_not_found = find_match_cosine(all_comp_np, comp_not_found_embs, all_competencies, comp_not_found)

In [220]:
df_sim_not_found

,Найденные компетенции,Существующие компетенции,cosine_sim
0,Разработка и контроль реализации маркетинговой стратегии компании,Разработка маркетинговой политики компании,0.768
1,"Составление, согласование и контроль исполнения маркетингового бюджета",Реализация маркетинговых и рекламных мероприятий,0.629
2,"Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж",Разработка ценовой политики компании,0.68
3,"Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей",Реализация маркетинговых и рекламных мероприятий,0.646
4,Контроль разработки фирменного стиля,Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.685
5,Оптимизация бизнес-процессов маркетинговой деятельности компании,"Создание, развитие и оптимизация функциональной, стратегической, операционной и организационной систем для выполнения маркетинговых функций в организации",0.7
6,Выстраивание системы взаимодействия маркетинговой службы с другими подразделениями организации,"Создание, развитие и оптимизация функциональной, стратегической, операционной и организационной систем для выполнения маркетинговых функций в организации",0.698
7,Планирование деятельности маркетинговой службы компании,Разработка маркетинговой политики компании,0.71
8,Планирование и разработка бюджета реализуемых маркетинговых проектов,Реализация маркетинговых и рекламных мероприятий,0.681
9,"Проведение тестирования инновационных товаров, услуг, брендов",Оценка привлекательности нового продукта,0.562


In [236]:
df_match_final = pd.concat([
    df_[
        ~df_["Найденные компетенции"].isin(df_sim_not_found["Найденные компетенции"].values)
    ].rename(columns = {"Sim_score": "cosine_sim"}),
    df_sim_not_found
])

In [237]:
set(df_sim_query["Найденные компетенции"].values) -\
set(df_match_final["Найденные компетенции"].values).intersection(df_sim_query["Найденные компетенции"].apply(lambda x: x.strip()).values)

{'Создание брендов и их внедрение на рынок'}

In [239]:
df_sim_query_compare_director = df_sim_query.merge(
    df_match_final,
    on = "Найденные компетенции",
    suffixes=[" cosine", " combined"],
    how = "left"
)

In [240]:
df_sim_query_compare_director

,Найденные компетенции,Существующие компетенции cosine,cosine_sim cosine,Существующие компетенции combined,cosine_sim combined
0,Разработка и контроль реализации маркетинговой стратегии компании,Разработка маркетинговой политики компании,0.768,Разработка маркетинговой политики компании,0.768
1,"Составление, согласование и контроль исполнения маркетингового бюджета",Реализация маркетинговых и рекламных мероприятий,0.629,Реализация маркетинговых и рекламных мероприятий,0.629
2,"Контроль проведения маркетинговых исследований (оценка емкости рынка, анализ тенденций рынка, конкурентной среды и др.)",Проводить маркетинговые исследования и анализировать результаты,0.679,Проводить маркетинговые исследования и анализировать результаты,0.679
3,Организация мониторинга и анализа рекламной активности конкурентов,Аналитика по информационному полю и конкурентам,0.624,Анализ конкурентной среды,0.585
4,"Оптимизация продуктовой линейки и ценовой политики компании, каналы продаж",Разработка ценовой политики компании,0.68,Разработка ценовой политики компании,0.68
5,"Организация, контроль и анализ эффективности маркетинговых кампаний",Реализация маркетинговых и рекламных мероприятий,0.676,Анализ и корректировка рекламных кампаний,0.717
6,"Разработка и реализация программ по повышению уровня узнаваемости компании, стимулированию продаж, повышению лояльности потребителей",Реализация маркетинговых и рекламных мероприятий,0.646,Реализация маркетинговых и рекламных мероприятий,0.646
7,Контроль разработки бренда компании,"Разработка, контроль и корректировка стратегии проведения контекстно-медийной кампании",0.598,Создание брендов и их внедрение на рынок,0.59
8,Контроль разработки фирменного стиля,Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.685,Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.685
9,"Контроль разработки полиграфической и сувенирной продукции, рекламно-информационных материалов в соответствии со стандартами фирменного стиля",Подготовка предложений по формированию фирменного стиля компании и фирменного стиля оформления рекламной продукции,0.664,Подготовка рекламных и информационных материалов,0.637


In [241]:
df_sim_query_compare_director.to_csv("./save/comp_matching_compare_director.csv", index=False)

In [245]:
df_sim_query_compare_director_ext = pd.read_csv("./save/comp_matching_compare_director_ext.csv")

In [249]:
df_sim_query_compare_director_ext["Соответствие"].value_counts(normalize=True).round(3).to_frame()

,proportion
Соответствие,
Хорошо,0.333
Идеально,0.278
Норм,0.278
Не соответствует,0.111


In [250]:
df_sim_query_compare_director_ext["Сравнение методов"].value_counts(normalize=True).round(3).to_frame()

,proportion
Сравнение методов,
Одинаково,0.528
Лучше cosine,0.361
Лучше combined,0.111
